In [56]:
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [57]:

import os
import dotenv , getpass

dotenv.load_dotenv()

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()


import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.prompts import PromptTemplate
import time
import requests
from bs4 import BeautifulSoup as soup
import urllib.parse
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import Html2TextTransformer


In [58]:

embeddings = (
    OllamaEmbeddings(model="all-minilm")
)  
llm = Ollama(model="gemma:7b")

In [59]:
question="Best pizza places in Cairo Egypt"

words = question.split()
url_search = '+'.join(words)
print(url_search)

seed_url=f'https://www.google.com/search?q={url_search}'
# google_maps_url=f'https://www.google.com/maps/search/{url_search}'
print('seed_url:',seed_url)
# print('google_maps_url:',google_maps_url)


general_fetched_urls=[]
# all_fetched_urls=[]

Best+pizza+places+in+Cairo+Egypt
seed_url: https://www.google.com/search?q=Best+pizza+places+in+Cairo+Egypt


In [60]:
# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/",
}
response = requests.get(seed_url, headers=headers)
if response.status_code == 200:
    page_content = response.text
    
    # Parse HTML content
    sp = soup(page_content, 'html.parser')
    
    # Find all <a> tags and extract URLs
     # Find all <a> tags and extract URLs
    for a_tag in sp.find_all('a', href=True):
        link = a_tag['href']

        # Check if the link contains 'url=' and '&ved'
        if 'url=' in link and '&ved=' in link:
            # Extract the part after 'url='
            url_part = link.split('url=')[1]
            # Find the position of '&ved=' and slice the URL up to that position
            url = url_part.split('&ved=')[0]
            # Decode the URL to handle any URL encoding
            decoded_url = urllib.parse.unquote(url)
            
            # Ensure the URL starts with 'https://'
            if decoded_url.startswith('https://'):
                print(decoded_url)
                # scraped_urls.append(decoded_url)
                try:
                    # Try to make a request to the URL to check for SSL certificate validity
                    r = requests.get(decoded_url, headers=headers, verify=True, timeout=20)
                    # If the request is successful and no SSL errors are raised, add the URL to the list
                    print(r.status_code)
                    if r.status_code == 200 or r.status_code == 520:
                        general_fetched_urls.append(decoded_url)
                except requests.exceptions.SSLError:
                    # SSL certificate is not valid
                    print(f"SSL Error for URL: {link}")
                except requests.exceptions.RequestException as e:
                    # Handle other request exceptions (timeouts, connection errors, etc.)
                    print(f"Request Exception for URL: {link}, Error: {e}")




https://maps.google.com/maps?q=Best+pizza+places+in+Cairo+Egypt&um=1&ie=UTF-8&ved=1t:200713&ictx=111&rct=j&q=&esrc=s&opi=89978449&sa=U
200
https://www.tripadvisor.com/Restaurants-g294201-c31-Cairo_Cairo_Governorate.html
200
https://top10cairo.com/best-pizza-cairo/
520
https://maps.google.com/maps?um=1&ie=UTF-8&fb=1&gl=eg&sa=X&q=Best+pizza+places+in+Cairo+Egypt&ved=1t:200245&ictx=111
200
https://sceneeats.com/Spots/A-SceneEats-Guide-to-the-Best-Authentic-Pizza-Spots-in-Cairo
200
https://www.egypttoday.com/Article/6/129900/Top-5-Neapolitan-Pizza-Places-in-Cairo
200
https://rahetbally.com/en/read/articles/best-pizza-places-in-cairo
200
https://tourteller.com/blog/best-pizza-places-in-cairo/
200
https://lovin.co/cairo/en/latest/top-pizza-spots-in-cairo-that-you-wont-want-to-miss/
200
https://www.talabat.com/egypt/cuisine/pizza
200
https://www.cairo360.com/article/restaurants/these-restaurants-are-serving-up-cairos-best-pizzas/
200
https://curlsenroute.com/pizza-in-cairo-a-quest-for-the-mos

In [61]:

# Display the scraped URLs
print('General fetched Urls:')
print(general_fetched_urls)
# all_fetched_urls.extend(general_fetched_urls)
# all_fetched_urls.extend(google_maps_url)

# print('All fetched Urls:')
# # print(all_fetched_urls)
print("---------------------------------")


General fetched Urls:
['https://maps.google.com/maps?q=Best+pizza+places+in+Cairo+Egypt&um=1&ie=UTF-8&ved=1t:200713&ictx=111&rct=j&q=&esrc=s&opi=89978449&sa=U', 'https://www.tripadvisor.com/Restaurants-g294201-c31-Cairo_Cairo_Governorate.html', 'https://top10cairo.com/best-pizza-cairo/', 'https://maps.google.com/maps?um=1&ie=UTF-8&fb=1&gl=eg&sa=X&q=Best+pizza+places+in+Cairo+Egypt&ved=1t:200245&ictx=111', 'https://sceneeats.com/Spots/A-SceneEats-Guide-to-the-Best-Authentic-Pizza-Spots-in-Cairo', 'https://www.egypttoday.com/Article/6/129900/Top-5-Neapolitan-Pizza-Places-in-Cairo', 'https://rahetbally.com/en/read/articles/best-pizza-places-in-cairo', 'https://tourteller.com/blog/best-pizza-places-in-cairo/', 'https://lovin.co/cairo/en/latest/top-pizza-spots-in-cairo-that-you-wont-want-to-miss/', 'https://www.talabat.com/egypt/cuisine/pizza', 'https://www.cairo360.com/article/restaurants/these-restaurants-are-serving-up-cairos-best-pizzas/', 'https://curlsenroute.com/pizza-in-cairo-a-ques

In [62]:
start_time = time.time()

In [63]:
loader =  AsyncChromiumLoader(general_fetched_urls, user_agent="MyAppUserAgent",)
docs=  loader.load()
print(len(docs))

Task exception was never retrieved
future: <Task finished name='Task-2' coro=<Connection.run() done, defined at e:\MahyDolphin\Gsoc-2024-lg\Coding_Period\REPO\LG-Gemma-AI-Touristic-info-tool\AI_Gemma_Model\gsoc_venv\lib\site-packages\playwright\_impl\_connection.py:265> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\Mahinour Elsarky\.pyenv\pyenv-win\versions\3.10.11\lib\asyncio\tasks.py", line 232, in __step
    result = coro.send(None)
  File "e:\MahyDolphin\Gsoc-2024-lg\Coding_Period\REPO\LG-Gemma-AI-Touristic-info-tool\AI_Gemma_Model\gsoc_venv\lib\site-packages\playwright\_impl\_connection.py", line 272, in run
    await self._transport.connect()
  File "e:\MahyDolphin\Gsoc-2024-lg\Coding_Period\REPO\LG-Gemma-AI-Touristic-info-tool\AI_Gemma_Model\gsoc_venv\lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "e:\MahyDolphin\Gsoc-2024-lg\Coding_Period\REPO\LG-Gemma-AI-Touristic-info-tool\AI_Gemma_Model\gsoc

NotImplementedError: 

In [ ]:
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)
# docs_transformed[0].page_content[0:500]
print(f'docs_transformed 0:{docs_transformed[0]}')
print(f'docs_transformed 1:{docs_transformed[1]}')
print(f'docs_transformed 2:{docs_transformed[2]}')

TypeError: 'coroutine' object is not iterable

In [ ]:


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs_transformed)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
print('vectorestore initialized')
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
print('retriever initialized')




In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [ ]:


prompt = PromptTemplate(
                                input_variables = ['question', 'context'], 
                                template=
                                '''
                                You are a helpful touristic advisor bot. Your name is Adventura.
                                You help people find the best places to visit, eat and stay nearby their location.
                                Can you find me 10 places for {question} and include all details like:
                                - name
                                - description
                                - genre
                                - rating
                                - opening hours
                                - closing hours
                                - accurate address_location
                                - Link to their website
                                - any interesting information you have on them 
                                using this extra context: {context}
                                '''
                                )


In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:

input_data = {
    'question': question,
}


In [ ]:

print('Asking the model:')
llm_results=rag_chain.invoke(input=input_data)
print(llm_results)


In [ ]:

elapsed_time = time.time() - start_time
print(f"Execution time: {elapsed_time:.2f} seconds")


In [ ]:

# cleanup
vectorstore.delete_collection()